In [1]:
from __future__ import division
from psychopy.visual import ImageStim, TextStim, Window
from psychopy import core, event, gui, data, logging
import numpy as np
import pandas as pd
import os

from routines import Routine

# Code for the (simplified) reinforcement learning task from Fontanesi and colleagues (https://doi.org/10.3758/s13423-018-1554-2)

#general settings
expName = 'Group A'
screen_size = [2000, 1200]
frames_per_second = 60
full_screen = False
background_color = '#eeeeee'
n_trials = 30

# trial settings
choice_keys = ['q', 'p']
escape_key = 'escape'
choice_time_limit = 5
feedback_duration = 2
fixation_duration = 1

#stimuli settings
text_color = 'black'
text_height = 50
options_x_offset = 200
image_size = 100

#store info about the experiment session
dlg = gui.Dlg(title=expName)
dlg.addField('Participant:', 1)
dlg.addField('Age:', 25)
dlg.addField('Gender:', choices=['female', 'male', 'prefer not to disclose'])
dlg.addField("condition", choices = ["A", "B"]) #change into A and B
dlg.addField('Handedness:', choices=['right', 'left', 'both'])
dlg.show()

expInfo = dict(zip(['participant', 'age', 'gender', 'hand'], dlg.data))
expInfo['date'] = data.getDateStr()  # add a simple timestamp
expInfo['expName'] = expName # add the experiment name
if dlg.OK:  # then the user pressed OK
    print(expInfo)
else:
    print(expInfo)
    core.quit()

#check if data folder exists
directory=os.path.join(os.getcwd(), 'data')
if not os.path.exists(directory):
    os.makedirs(directory)

#create file name for storing data
fileName = os.path.join('data', '%s_%s_%s' % (expName, expInfo['participant'], expInfo['date']))

#save a log file
logFile = logging.LogFile(fileName + '.log', level=logging.DEBUG)
logging.console.setLevel(logging.WARNING)  # this outputs to the screen, not a file

#create a window
mywin = Window(screen_size, units='pix', color=background_color, fullscr=full_screen)

#create some stimuli
left_feedback = TextStim(win=mywin, color=text_color, pos=(-options_x_offset, 0), height=text_height)
right_feedback = TextStim(win=mywin, color=text_color, pos=(options_x_offset, 0), height=text_height)

left_stimuli = TextStim(win=mywin, pos=(-options_x_offset, 0), height=text_height)
right_stimuli = TextStim(win=mywin, pos=(options_x_offset, 0), height=text_height)

#Lotterien (2x2 outcomes, ähnlichem EV, verschiedene Varianz)
#Zufällige Zahlen 1 bis 10 x 30
a = np.random.choice(a = np.arange(1,11), size = 40, replace = True)

#Zufällige Wahrscheinlichkeiten x 30 (px)
b = np.random.choice(a = np.arange(0.3, 0.7, 0.05), size = 40, replace = True)

#EVx: x * px
c = a * b

# Y = evx/py
d = c / (1 - b)

#py: 1 - px
e = 1 - b

#Evy: y * py
f = d * e

#creating data frame for nornal Lotteries
x = {"x": a, "px": b, "evx": c, "y": d, "py": e, "evy": f}
normallot = pd.DataFrame(x)

#Lotterien (plus 1000 bei outcomes) #within subject
# Lotterie mit grossen Outcomes
# x + 1000
abig = a + 1000
# px (gleich wie bei normal lotteries)
bbig = b
# evx = abig * bbig
cbig = abig * bbig
# Y = evx/py
dbig = cbig / (1 - bbig)
# py = 1 - px
ebig = 1 - bbig
# evy = y * py
fbig = dbig * ebig
#creating data frame
x = {"x": abig, "px": bbig, "evx": cbig, "y": dbig, "py": ebig, "evy": fbig}
biglot = pd.DataFrame(x)

#Lotterien mit rare event 
#Zufällige Zahlen 1 bis 10 x 30
ahard = np.random.choice(a = np.arange(1,11), size = 40, replace = True)

#Zufällige Wahrscheinlichkeiten x 30 (px zwieschen 0.1 und 0.3)
bhard = np.random.choice(a = np.arange(0.1, 0.3, 0.05), size = 40, replace = True)

#EVx: x * px
chard = ahard * bhard

# Y = evx/py
dhard = chard / (1 - bhard)

#py: 1 - px
ehard = 1 - bhard

#Evy: y * py
fhard = dhard * ehard

#creating data frame for nornal Lotteries
x = {"x": ahard, "px": bhard, "evx": chard, "y": dhard, "py": ehard, "evy": fhard}
hardlot = pd.DataFrame(x)
hardlot

#Lotterien mit hohen outcomes und rare events
#Zufällige Zahlen 1 bis 10 x 30
abigh = abig

#Zufällige Wahrscheinlichkeiten x 30 (px zwieschen 0.1 und 0.3)
bbigh = b

#EVx: x * px
cbigh = abigh * bbigh

# Y = evx/py
dbigh = cbigh / (1 - bbigh)

#py: 1 - px
ebigh = 1 - bbigh

#Evy: y * py
fbigh = dbigh * ebigh

#creating data frame for nornal Lotteries
x = {"x": abigh, "px": bbigh, "evx": cbigh, "y": dbigh, "py": ebigh, "evy": fbigh}
bighlot = pd.DataFrame(x)

#fixation cross
fixation_cross = TextStim(win=mywin, text='+', color=text_color)

#create the dataframe
data = pd.DataFrame([])

#draw the stimuli
trial_routine = Routine(window=mywin, frames_per_second=frames_per_second, escape_key=escape_key)


for t in range(n_trials):
    # put here things that change every trial$
    if "condition" == "A": #A = easy option/condition
        stimuli = {'left_stimuli': [normallot.loc[t, ["x", "px", "y", "py"]] , biglot.loc[t, ["x", "px", "y", "py"]]],
         'right_stimuli': [biglot.loc[t, ["x", "px", "y", "py"]], normallot.loc[t, ["x", "px", "y", "py"]]],
         'left_feedback': [np.random.choice(a = normallot.loc[t, ["x", "y"]],  p= normallot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = biglot.loc[t, ["x", "y"]],  p= biglot.loc[t, ["px", "py"]])], 
         'right_feedback': [np.random.choice(a = biglot.loc[t, ["x", "y"]],  p= biglot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = normallot.loc[t, ["x", "y"]],  p= normallot.loc[t, ["px", "py"]])]
        }
    else:
         stimuli = {'left_stimuli': [hardlot.loc[t, ["x", "px", "y", "py"]] , bighlot.loc[t, ["x", "px", "y", "py"]]],
         'right_stimuli': [bighlot.loc[t, ["x", "px", "y", "py"]], hardlot.loc[t, ["x", "px", "y", "py"]]],
         'left_feedback': [np.random.choice(a = hardlot.loc[t, ["x", "y"]],  p= hardlot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = bighlot.loc[t, ["x", "y"]],  p= bighlot.loc[t, ["px", "py"]])], 
         'right_feedback': [np.random.choice(a = bighlot.loc[t, ["x", "y"]],  p= bighlot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = hardlot.loc[t, ["x", "y"]],  p= hardlot.loc[t, ["px", "py"]])]
        }
    randomizer = np.random.choice(a = [0,1])
    if randomizer == 0:
        left_stimuli.text = "%s" % stimuli["left_stimuli"] 
        left_feedback.text = '%s' % stimuli["left_feedback"]
        right_stimuli.text = "%s" % stimuli["right_stimuli"]
        right_feedback.text = '%s' % stimuli["right_feedback"]
    else:
        left_stimuli.text = "%s" % stimuli["left_stimuli"] 
        left_feedback.text = '%s' % stimuli["left_feedback"]
        right_stimuli.text = "%s" % stimuli["right_stimuli"]
        right_feedback.text = '%s' % stimuli["right_feedback"]
 
      
    # first event
    trial_routine.wait_for_time_limit(
        components=[fixation_cross], 
        time_seconds=fixation_duration, 
        label='fixation_cross')
    
    # second event
    #antwort registrieren und speichern
    key, rt = trial_routine.wait_for_keys(
        components=[left_stimuli, right_stimuli],
        valid_keys=choice_keys,
        label='gamble_choice')
    
    data = data.append({'trial': t, "choice": key, "f_left": left_feedback, "f_right" : right_feedback
    }, ignore_index=True)
    
    # third event 
    trial_routine.wait_for_time_limit(
        components=[left_feedback, right_feedback], 
        time_seconds=feedback_duration, 
        label= "feedback")
    
    
       
    #save data to file
    for label in expInfo.keys():
        data[label] = expInfo[label]
    data.to_csv(fileName + '.csv')
    
#cleanup
mywin.close()
core.quit()

pygame 2.0.1 (SDL 2.0.14, Python 3.6.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\mathi\anaconda3\envs\psychopy\lib\site-packages\pyglet\media\codecs\wmf.py:838: UserWarning: [WinError -2147417850] Der Threadmodus kann nicht nach dem Einstellen geändert werden
  warnings.warn(str(err))


{'participant': 1, 'age': 25, 'gender': 'female', 'hand': 'A', 'date': '2021_Jun_15_1706', 'expName': 'Group A'}


C:\Users\mathi\anaconda3\envs\psychopy\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Der Threadmodus kann nicht nach dem Einstellen geändert werden
  warnings.warn(str(err))


9.9117 	WARNING 	Monitor specification not found. Creating a temporary one...
14.0899 	WARNING 	Couldn't measure a consistent frame rate.
  - Is your graphics card set to sync to vertical blank?
  - Are you running other processes on your computer?

14.4676 	WARNING 	t of last frame was 378.38ms (=1/2)
14.5753 	WARNING 	t of last frame was 37.10ms (=1/26)
14.6092 	WARNING 	t of last frame was 33.86ms (=1/29)
14.6377 	WARNING 	t of last frame was 28.49ms (=1/35)
14.6666 	WARNING 	Multiple dropped frames have occurred - I'll stop bothering you about them!


SystemExit: 0

C:\Users\mathi\anaconda3\envs\psychopy\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import numpy as np
import pandas as pd
t = 5
if "condition" == "A": #A = easy option/condition
        stimuli = pd.DataFrame(
        {'trial_type': ["normal-big", "big-normal"],
         'left_stimuli': [normallot.loc[t, ["x", "px", "y", "py"]] , biglot.loc[t, ["x", "px", "y", "py"]]],
         'right_stimuli': [biglot.loc[t, ["x", "px", "y", "py"]], normallot.loc[t, ["x", "px", "y", "py"]]],
         'left_feedback': [np.random.choice(a = normallot.loc[t, ["x", "y"]],  p= normallot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = biglot.loc[t, ["x", "y"]],  p= biglot.loc[t, ["px", "py"]])], 
         'right_feedback': [np.random.choice(a = biglot.loc[t, ["x", "y"]],  p= biglot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = normallot.loc[t, ["x", "y"]],  p= normallot.loc[t, ["px", "py"]])]
        })
else:
         stimuli = pd.DataFrame(
        {'trial_type': ["hard-bighard", "bighard-big"],
         'left_stimuli': [hardlot.loc[t, ["x", "px", "y", "py"]] , bighlot.loc[t, ["x", "px", "y", "py"]]],
         'right_stimuli': [bighlot.loc[t, ["x", "px", "y", "py"]], hardlot.loc[t, ["x", "px", "y", "py"]]],
         'left_feedback': [np.random.choice(a = hardlot.loc[t, ["x", "y"]],  p= hardlot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = bighlot.loc[t, ["x", "y"]],  p= bighlot.loc[t, ["px", "py"]])], 
         'right_feedback': [np.random.choice(a = bighlot.loc[t, ["x", "y"]],  p= bighlot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = hardlot.loc[t, ["x", "y"]],  p= hardlot.loc[t, ["px", "py"]])]
        }) 
randomizer = np.random.choice(a= [0,1])
if randomizer == 0:
    stimuli = stimuli[0:["x", "px"]] 
else:
    stimuli = stimuli[1:]
    
print(stimuli[0:])


NameError: name 'hardlot' is not defined

In [87]:
stimuli = pd.DataFrame(
    {'trial_type': ["normal-big", "big-normal"],
     'left_stimuli': [normallot.loc[t, ["x", "px", "y", "py"]] , biglot.loc[t, ["x", "px", "y", "py"]]],
     'right_stimuli': [biglot.loc[t, ["x", "px", "y", "py"]], normallot.loc[t, ["x", "px", "y", "py"]]],
     'left_feedback': [np.random.choice(a = normallot.loc[t, ["x", "y"]],  p= normallot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = biglot.loc[t, ["x", "y"]],  p= biglot.loc[t, ["px", "py"]])], 
     'right_feedback': [np.random.choice(a = biglot.loc[t, ["x", "y"]],  p= biglot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = normallot.loc[t, ["x", "y"]],  p= normallot.loc[t, ["px", "py"]])]
    })
stimuli

,trial_type,left_stimuli,right_stimuli,left_feedback,right_feedback
0,normal-big,x 5.000000 px 0.400000 y 3.333333 p...,x 1007.000000 px 0.400000 y 671...,5.0,671.333333
1,big-normal,x 1007.000000 px 0.400000 y 671...,x 5.000000 px 0.400000 y 3.333333 p...,1007.0,3.333333


In [104]:
stimuli.loc[1:]

,trial_type,left_stimuli,right_stimuli,left_feedback,right_feedback
1,big-normal,x 1007.000000 px 0.400000 y 671...,x 5.000000 px 0.400000 y 3.333333 p...,1007.0,3.333333


In [54]:
asdfgghh = np.tile([np.random.choice(a = normallot.loc[t, ["x", "y"]],  p= normallot.loc[t, ["px", "py"]]), 
                            np.random.choice(a = biglot.loc[t, ["x", "y"]],  p= biglot.loc[t, ["px", "py"]])], 15)
asdfgghh

array([   3.33333333, 1007.        ,    3.33333333, 1007.        ,
          3.33333333, 1007.        ,    3.33333333, 1007.        ,
          3.33333333, 1007.        ,    3.33333333, 1007.        ,
          3.33333333, 1007.        ,    3.33333333, 1007.        ,
          3.33333333, 1007.        ,    3.33333333, 1007.        ,
          3.33333333, 1007.        ,    3.33333333, 1007.        ,
          3.33333333, 1007.        ,    3.33333333, 1007.        ,
          3.33333333, 1007.        ])

In [121]:
np.tile(['normalleft', 'normalright', 'bigleft', 'bigright'], 7.5)

TypeError: 'float' object cannot be interpreted as an integer

In [5]:
#Lotteriensammlung
import numpy as np
import pandas as pd
#Zufällige Zahlen 1 bis 10 x 30
a = np.random.choice(a = np.arange(1,11), size = 30, replace = True)

#Zufällige Wahrscheinlichkeiten x 30 (px)
b = np.random.choice(a = np.arange(0.3, 0.7, 0.05), size = 30, replace = True)

#EVx: x * px
c = a * b

# Y = evx/py
d = c / (1 - b)

#py: 1 - px
e = 1 - b

#Evy: y * py
f = d * e

#varx
g = {"x": a, "px": b}
h = pd.DataFrame(g)
i = np.var(h, axis = 1)

#vary
j = {"y": d, "py": e}
k = pd.DataFrame(j)
l = np.var(k, axis = 1)

#creating data frame for nornal Lotteries
x = {"x": a, "px": b, "evx": c, "varx": i, "y": d, "py": e, "evy": f, "vary": l}
normallot = pd.DataFrame(x)
normallot

,x,px,evx,varx,y,py,evy,vary
0,1,0.65,0.65,0.030625,1.857143,0.35,0.65,0.567870
1,10,0.55,5.50,22.325625,12.222222,0.45,5.50,34.646304
2,4,0.50,2.00,3.062500,4.000000,0.50,2.00,3.062500
3,8,0.60,4.80,13.690000,12.000000,0.40,4.80,33.640000
4,4,0.55,2.20,2.975625,4.888889,0.45,2.20,4.925934
5,2,0.40,0.80,0.640000,1.333333,0.60,0.80,0.134444
6,7,0.50,3.50,10.562500,7.000000,0.50,3.50,10.562500
7,9,0.65,5.85,17.430625,16.714286,0.35,5.85,66.947462
8,1,0.50,0.50,0.062500,1.000000,0.50,0.50,0.062500
9,9,0.65,5.85,17.430625,16.714286,0.35,5.85,66.947462


In [7]:
# Lotterie mit grossen Outcomes
# x + 1000
abig = np.random.choice(a = np.arange(1,11), size = 30, replace = True) + 1000
# px (gleich wie bei normal lotteries)
bbig = b
# evx = abig * bbig
cbig = abig * bbig
# Y = evx/py
dbig = cbig / (1 - bbig)
# py = 1 - px
ebig = 1 - bbig
# evy = y * py
fbig = dbig * ebig
#varx
gbig = {"x": abig, "px": b}
hbig = pd.DataFrame(gbig)
ibig = np.var(hbig, axis = 1)

#vary
jbig = {"y": dbig, "py": ebig}
kbig = pd.DataFrame(jbig)
lbig = np.var(kbig, axis = 1)
#creating data frame
x = {"x": abig, "px": bbig, "evx": cbig, "varx": ibig, "y": dbig, "py": ebig, "evy": fbig, "vary": lbig}
biglot = pd.DataFrame(x)
biglot

,x,px,evx,varx,y,py,evy,vary
0,1009,0.60,605.40,254217.640000,1513.500000,0.40,605.40,572367.902500
1,1008,0.40,403.20,253814.440000,672.000000,0.60,403.20,112694.490000
2,1006,0.60,603.60,252707.290000,1509.000000,0.40,603.60,568968.490000
3,1005,0.50,502.50,252255.062500,1005.000000,0.50,502.50,252255.062500
4,1002,0.45,450.90,250775.600625,819.818182,0.55,450.90,167800.088435
5,1009,0.65,655.85,254192.430625,1873.857143,0.35,655.85,877507.253584
6,1004,0.30,301.20,251853.422500,430.285714,0.70,301.20,46135.971480
7,1008,0.45,453.60,253789.250625,824.727273,0.55,453.60,169817.044220
8,1002,0.45,450.90,250775.600625,819.818182,0.55,450.90,167800.088435
9,1004,0.40,401.60,251803.240000,669.333333,0.60,401.60,111801.067778


In [6]:
# schwere Lotterie
#Zufällige Zahlen 1 bis 10 x 30
ahard = np.random.choice(a = np.arange(1,11), size = 30, replace = True)

#Zufällige Wahrscheinlichkeiten x 30 (px zwieschen 0.1 und 0.3)
bhard = np.random.choice(a = np.arange(0.1, 0.3, 0.05), size = 30, replace = True)

#EVx: x * px
chard = ahard * bhard

# Y = evx/py
dhard = chard / (1 - bhard)

#py: 1 - px
ehard = 1 - bhard

#Evy: y * py
fhard = dhard * ehard

#varx
ghard = {"x": ahard, "px": bhard}
hhard = pd.DataFrame(ghard)
ihard = np.var(hhard, axis = 1)

#vary
jhard = {"y": dhard, "py": ehard}
khard = pd.DataFrame(jhard)
lhard = np.var(khard, axis = 1)

#creating data frame for nornal Lotteries
x = {"x": ahard, "px": bhard, "evx": chard, "varx": ihard, "y": dhard, "py": ehard, "evy": fhard, "vary": lhard}
hardlot = pd.DataFrame(x)
hardlot

,x,px,evx,varx,y,py,evy,vary
0,3,0.10,0.30,2.102500,0.333333,0.90,0.30,0.080278
1,6,0.10,0.60,8.702500,0.666667,0.90,0.60,0.013611
2,3,0.20,0.60,1.960000,0.750000,0.80,0.60,0.000625
3,1,0.15,0.15,0.180625,0.176471,0.85,0.15,0.113410
4,9,0.25,2.25,19.140625,3.000000,0.75,2.25,1.265625
5,2,0.20,0.40,0.810000,0.500000,0.80,0.40,0.022500
6,6,0.25,1.50,8.265625,2.000000,0.75,1.50,0.390625
7,1,0.25,0.25,0.140625,0.333333,0.75,0.25,0.043403
8,3,0.10,0.30,2.102500,0.333333,0.90,0.30,0.080278
9,6,0.15,0.90,8.555625,1.058824,0.85,0.90,0.010902


In [7]:
# schwere + big Lotterie
#Zufällige Zahlen 1 bis 10 x 30
abigh = abig

#Zufällige Wahrscheinlichkeiten x 30 (px zwieschen 0.1 und 0.3)
bbigh = b

#EVx: x * px
cbigh = abigh * bbigh

# Y = evx/py
dbigh = cbigh / (1 - bbigh)

#py: 1 - px
ebigh = 1 - bbigh

#Evy: y * py
fbigh = dbigh * ebigh

#varx
gbigh = {"x": abigh, "px": bbigh}
hbigh = pd.DataFrame(gbigh)
ibigh = np.var(hbigh, axis = 1)

#vary
jbigh = {"y": dbigh, "py": ebigh}
kbigh = pd.DataFrame(jbigh)
lbigh = np.var(kbigh, axis = 1)

#creating data frame for nornal Lotteries
x = {"x": abigh, "px": bbigh, "evx": cbigh, "varx": ibigh, "y": dbigh, "py": ebigh, "evy": fbigh, "vary": lbigh}
bighlot = pd.DataFrame(x)
bighlot

NameError: name 'abig' is not defined